In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_stones"
cohort = "GSE123993"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_stones"
in_cohort_dir = "../../input/GEO/Kidney_stones/GSE123993"

# Output paths
out_data_file = "../../output/preprocess/Kidney_stones/GSE123993.csv"
out_gene_data_file = "../../output/preprocess/Kidney_stones/gene_data/GSE123993.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_stones/clinical_data/GSE123993.csv"
json_path = "../../output/preprocess/Kidney_stones/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"No effect of calcifediol supplementation on skeletal muscle transcriptome in vitamin D deficient frail older adults."
!Series_summary	"Vitamin D deficiency is common among older adults and has been linked to muscle weakness. Vitamin D supplementation has been proposed as a strategy to improve muscle function in older adults. The aim of this study was to investigate the effect of calcifediol (25-hydroxycholecalciferol) on whole genome gene expression in skeletal muscle of vitamin D deficient frail older adults. A double-blind placebo controlled trial was conducted in vitamin D deficient frail older adults (aged above 65), characterized by blood 25-hydroxycholecalciferol concentrations between 20 and 50 nmol/L. Subjects were randomized across the placebo group (n=12) and the calcifediol group (n=10, 10 µg per day). Muscle biopsies were obtained before and after six months of calcifediol or placebo supplementation and subjected to whole genome gene e

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze dataset and extract clinical features

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from skeletal muscle biopsies
# Affymetrix HuGene 2.1ST arrays were used, which indicates this is gene expression microarray data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Identify keys in sample characteristics dictionary for trait, age, and gender

# For trait: Intervention group as trait (calcifediol vs placebo)
trait_row = 3  # 'intervention group: 25-hydroxycholecalciferol (25(OH)D3)' or 'intervention group: Placebo'

# For gender: Sex information is available
gender_row = 1  # 'Sex: Male' or 'Sex: Female'

# For age: No specific age information available in the sample characteristics
age_row = None  # Age data is not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value (intervention group) to binary format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for 25-hydroxycholecalciferol, 0 for Placebo
    if '25-hydroxycholecalciferol' in value or '25(OH)D3' in value:
        return 1
    elif 'Placebo' in value:
        return 0
    else:
        return None

def convert_gender(value):
    """Convert gender value to binary format (0 for Female, 1 for Male)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

def convert_age(value):
    """This function is not used since age data is not available."""
    return None

# 3. Save metadata - conduct initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical data extraction
if trait_row is not None:
    # We need to create a DataFrame from the sample characteristics dictionary
    # The dictionary structure matches what would be expected by geo_select_clinical_features
    
    # Sample characteristics data from previous step
    sample_chars_dict = {0: ['tissue: muscle'], 
                        1: ['Sex: Male', 'Sex: Female'], 
                        2: ['subject id: 3087', 'subject id: 3088', 'subject id: 3090', 'subject id: 3106', 
                            'subject id: 3178', 'subject id: 3241', 'subject id: 3258', 'subject id: 3279', 
                            'subject id: 3283', 'subject id: 3295', 'subject id: 3322', 'subject id: 3341', 
                            'subject id: 3360', 'subject id: 3361', 'subject id: 3375', 'subject id: 3410', 
                            'subject id: 3430', 'subject id: 3498', 'subject id: 3516', 'subject id: 3614', 
                            'subject id: 3695', 'subject id: 3731'], 
                        3: ['intervention group: 25-hydroxycholecalciferol (25(OH)D3)', 'intervention group: Placebo'], 
                        4: ['time of sampling: before intervention (baseline)', 'time of sampling: after intervention']}
    
    # Convert dictionary to DataFrame for processing
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender,
        age_row=age_row,
        convert_age=convert_age
    )
    
    # Preview the selected clinical features
    print("Selected Clinical Features Preview:")
    print(preview_df(selected_clinical))
    
    # Save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Selected Clinical Features Preview:
{0: [1.0, 1.0], 1: [0.0, 0.0], 2: [nan, nan], 3: [nan, nan], 4: [nan, nan], 5: [nan, nan], 6: [nan, nan], 7: [nan, nan], 8: [nan, nan], 9: [nan, nan], 10: [nan, nan], 11: [nan, nan], 12: [nan, nan], 13: [nan, nan], 14: [nan, nan], 15: [nan, nan], 16: [nan, nan], 17: [nan, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan], 21: [nan, nan]}
Clinical data saved to ../../output/preprocess/Kidney_stones/clinical_data/GSE123993.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Kidney_stones/GSE123993/GSE123993_series_matrix.txt.gz


Gene data shape: (53617, 44)
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene/probe identifiers observed, these appear to be probe IDs or some other numeric identifiers
# rather than standard human gene symbols (which would typically be letters like BRCA1, TP53, etc.)
# Therefore, these identifiers would need to be mapped to human gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the gene annotation columns
print("\nAnalyzing gene symbol related columns:")
if 'GENE_SYMBOL' in gene_annotation.columns:
    print(f"Sample GENE_SYMBOL values: {gene_annotation['GENE_SYMBOL'].dropna().head(5).tolist()}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0]
print(f"\nGene data first ID: {gene_data_id_prefix}")

# Look for columns that might contain probe IDs
print("\nAnalyzing potential probe ID columns:")
if 'ID' in gene_annotation.columns:
    print(f"Sample ID values: {gene_annotation['ID'].head(5).tolist()}")
    
if 'SPOT_ID' in gene_annotation.columns:
    print(f"Sample SPOT_ID values: {gene_annotation['SPOT_ID'].head(5).tolist()}")

# Check if there's any match between gene data index and annotation IDs
print("\nChecking for overlap between gene data IDs and annotation:")
gene_data_ids = set(gene_data.index[:1000])  # Get a sample of gene data IDs
annotation_ids = set(gene_annotation['ID'].astype(str)[:1000])
overlap = gene_data_ids.intersection(annotation_ids)
print(f"Number of IDs that match between gene data and annotation 'ID' column: {len(overlap)}")
print(f"Sample overlapping IDs: {list(overlap)[:5] if overlap else 'None'}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DE

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the gene annotation data preview, we need to identify which columns correspond to 
# probe IDs and gene symbols

# Looking at the structure of both datasets:
# 1. gene_data has IDs like '16650001', '16650003', etc. which appear to be numeric IDs
# 2. The gene_annotation dataframe has an 'ID' column with values like '16657436', which is similar in format
# 3. The 'gene_assignment' column contains gene symbols like 'DDX11L1', 'MIR1302-11', 'OR4F5'

# So we'll map from 'ID' to extracted gene symbols from 'gene_assignment'

# 1. Examine the 'ID' column and 'gene_assignment' column more closely
print("\nProbe ID column (first 5 values):")
print(gene_annotation['ID'].head(5).tolist())

# Examine what the gene_assignment column contains to determine how to extract gene symbols
print("\nGene assignment column sample:")
gene_sample = gene_annotation['gene_assignment'].dropna().iloc[0]
print(gene_sample[:200] + "..." if len(str(gene_sample)) > 200 else gene_sample)

# 2. Create gene mapping dataframe using the 'ID' and 'gene_assignment' columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print("\nGene mapping dataframe (first few rows):")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many mappings by equally dividing probe values among genes
gene_expression_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene expression data after mapping (shape):")
print(gene_expression_data.shape)
print("First few gene symbols:")
print(gene_expression_data.index[:10].tolist())

# Normalize the gene symbols to have consistent naming
gene_data = normalize_gene_symbols_in_index(gene_expression_data)
print("\nAfter normalizing gene symbols:")
print(f"Shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {gene_data.index[:10].tolist()}")

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Probe ID column (first 5 values):
['16657436', '16657440', '16657445', '16657447', '16657450']

Gene assignment column sample:
NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288...

Gene mapping dataframe (first few rows):
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (


Gene expression data after mapping (shape):
(81076, 44)
First few gene symbols:
['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1']

After normalizing gene symbols:
Shape: (23274, 44)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1', 'A4GALT']



Gene expression data saved to ../../output/preprocess/Kidney_stones/gene_data/GSE123993.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to Randall's plaque tissue, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data from Randall's plaque tissue associated with kidney stones."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for kidney stones analysis."
    )
    
    print("Dataset is not usable for kidney stones analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (23274, 44)
Gene data shape after normalization: (23274, 44)


Normalized gene expression data saved to ../../output/preprocess/Kidney_stones/gene_data/GSE123993.csv
Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   
3  !Sample_characteristics_ch1   
4  !Sample_characteristics_ch1   

                                          GSM3518336  \
0                                     tissue: muscle   
1                                          Sex: Male   
2                                   subject id: 3087   
3  intervention group: 25-hydroxycholecalciferol ...   
4   time of sampling: before intervention (baseline)   

                                          GSM3518337  \
0                                     tissue: muscle   
1                                          Sex: Male   
2                                   subject id: 3087   
3  intervention group: 25-hydroxycholecalciferol ...   
4               time of sampling: after int

Data shape after handling missing values: (44, 23276)
For the feature 'Kidney_stones', the least common label is '1.0' with 20 occurrences. This represents 45.45% of the dataset.
For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 45.45% of the dataset.
A new JSON file was created at: ../../output/preprocess/Kidney_stones/cohort_info.json


Linked data saved to ../../output/preprocess/Kidney_stones/GSE123993.csv
